<a href="https://colab.research.google.com/github/malakabdelbaki/Hybrid-Retrieval-Augmented-Generation-for-Structured-Financial-Data-Analysis/blob/main/RAG.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [17]:
import pandas as pd
from sentence_transformers import SentenceTransformer, CrossEncoder
import faiss
import numpy as np
from transformers import pipeline

df = pd.read_csv("/content/drive/MyDrive/TPCDS-50K.csv")

df["first_name"] = df["first_name"].fillna("Unknown")
df["income_lower"] = pd.to_numeric(df["income_lower"], errors='coerce')
df["income_upper"] = pd.to_numeric(df["income_upper"], errors='coerce')

def build_text(row):
    return (
        f"Client {row['CLIENT_ID']}: {row['first_name']} {row['last_name']}, "
        f"Age {row['age']}, Gender: {row['gender']}, Marital: {row['marital_status']}, "
        f"Education: {row['education']}, Income ${row['income_lower']:,.0f}-${row['income_upper']:,.0f}, "
        f"Credit: {row['credit_rating']}, Dependents: {row['num_dependants']}, "
        f"Buy potential: {row['household_buy_potential']}"
    )

df["text_to_embed"] = df.apply(build_text, axis=1)


In [18]:
embedding_model = SentenceTransformer('all-mpnet-base-v2')
all_embeddings = embedding_model.encode(df["text_to_embed"].tolist(), show_progress_bar=True, convert_to_numpy=True)

faiss_index = faiss.IndexFlatL2(all_embeddings.shape[1])
faiss_index.add(all_embeddings)

cross_encoder = CrossEncoder('cross-encoder/ms-marco-MiniLM-L-6-v2')

Batches:   0%|          | 0/1563 [00:00<?, ?it/s]

In [19]:
def retrieve_and_rerank(query, top_k=10):
    query_embedding = embedding_model.encode([query], convert_to_numpy=True)
    _, indices = faiss_index.search(query_embedding, min(top_k * 5, len(df)))
    candidates = df.iloc[indices[0]].copy()

    pairs = [(query, c) for c in candidates["text_to_embed"]]
    scores = cross_encoder.predict(pairs)
    candidates["score"] = scores
    return candidates.sort_values(by="score", ascending=False).head(top_k)


In [53]:
qa_pipeline = pipeline(
    "text2text-generation",
    model="MBZUAI/LaMini-Flan-T5-783M",
    device=0
)


Device set to use cuda:0


In [54]:
def format_record_as_bullet(record):
    return (
        f"- Name: {record['first_name']} {record['last_name']}\n"
        f"  Age: {record['age']}\n"
        f"  Gender: {record['gender']}\n"
        f"  Marital Status: {record['marital_status']}\n"
        f"  Education: {record['education']}\n"
        f"  Income: ${record['income_lower']:,.0f} - ${record['income_upper']:,.0f}\n"
        f"  Credit Rating: {record['credit_rating']}\n"
        f"  Dependents: {record['num_dependants']}\n"
        f"  Buy Potential: {record['household_buy_potential']}\n"
    )


def rag_response(query, top_k=3):
    records = retrieve_and_rerank(query, top_k)
    if records.empty:
        return "No relevant records found."

    formatted_context = "\n".join([format_record_as_bullet(row) for _, row in records.iterrows()])
    # prompt = f"Context:\n{formatted_context}\n\nQuestion: {query}\nAnswer:"
    prompt = (
    f"Context:\n{formatted_context}\n\n"
    f"Question: {query}\n"
    f"Only list the full names of the matching clients.\n"
    f"Answer:"
  )
    response = qa_pipeline(prompt, max_new_tokens=200)[0]['generated_text']
    return response


In [75]:
def find_client_by_name(df, first_name, last_name):
    return df[
        (df['first_name'].str.strip().str.lower() == first_name.strip().lower()) &
        (df['last_name'].str.strip().str.lower() == last_name.strip().lower())
    ]


In [55]:
query = "Who are two clients with a College education?"
print(rag_response(query))

John Creech and Ashley Murphy are two clients with a College education.


In [56]:
query = "Who are three clients with a College education?"
print(rag_response(query))

Alma Sharkey, John Creech, and Kenneth Adams.


In [77]:
find_client_by_name(df, "John", "Creech")

,CLIENT_ID,first_name,last_name,age,gender,marital_status,education,purchase_estimate,credit_rating,num_dependants,num_dependants_employed,num_dependants_count,household_income,income_lower,income_upper,household_buy_potential,text_to_embed
52,53,John,Creech,32.0,M,W,College,9000,Good,3,0,3,1,0,10000,Unknown,Client 53: John Creech ...


In [78]:
find_client_by_name(df, "Alma", "Sharkey")

,CLIENT_ID,first_name,last_name,age,gender,marital_status,education,purchase_estimate,credit_rating,num_dependants,num_dependants_employed,num_dependants_count,household_income,income_lower,income_upper,household_buy_potential,text_to_embed
2,3,Alma,Sharkey,64.0,M,S,College,2000,Good,1,5,6,18,170001,180000,1001-5000,Client 3: Alma Sharkey ...


In [79]:
find_client_by_name(df, "Kenneth", "Adams")

,CLIENT_ID,first_name,last_name,age,gender,marital_status,education,purchase_estimate,credit_rating,num_dependants,num_dependants_employed,num_dependants_count,household_income,income_lower,income_upper,household_buy_potential,text_to_embed
11,12,Kenneth,Adams,36.0,F,S,College,4000,Good,3,4,0,11,100001,110000,1001-5000,Client 12: Kenneth Adams ...


In [58]:
query = "List two elderly clients"
print(rag_response(query))

Richard Elder and Roger Hayes.


In [80]:
find_client_by_name(df, "Richard", "Elder")

,CLIENT_ID,first_name,last_name,age,gender,marital_status,education,purchase_estimate,credit_rating,num_dependants,num_dependants_employed,num_dependants_count,household_income,income_lower,income_upper,household_buy_potential,text_to_embed
27470,27471,Richard,Elder,79.0,F,U,Primary,1500,High Risk,4,0,5,20,190001,200000,501-1000,Client 27471: Richard Elder ...


In [81]:
find_client_by_name(df, "Roger", "Hayes")

,CLIENT_ID,first_name,last_name,age,gender,marital_status,education,purchase_estimate,credit_rating,num_dependants,num_dependants_employed,num_dependants_count,household_income,income_lower,income_upper,household_buy_potential,text_to_embed
92,93,Roger,Hayes,91.0,M,W,Advanced Degree,5000,Low Risk,2,5,1,16,150001,160000,Unknown,Client 93: Roger Hayes ...
